# *WARNING! Widgets are not implemented here. Instead, try the [Google Colab]([link](https://colab.research.google.com/drive/10GF0RY5FSCQh4VH0lTjjakNy284A4_Sw?usp=sharing#scrollTo=oKXUyZPxD5zQ) "YouTubeAudioDownloader") version of this notebook.*

# README FIRST

> **Step 1:** Click the `Connect` button from the top right side. (shown in the image below)

![image-1.png](https://cdn.discordapp.com/attachments/806892281360809985/1123589747420442654/image-1.png "Connect the program")


> **Step 2:** After it is connected, scroll down to the `Main Section` and fill-in the `User's FORM` part. The `VIDEO_LINK` is required and `ADD_MUSIC_METADATA` is optional. (shown in the image below)

![image-2.png](https://cdn.discordapp.com/attachments/806892281360809985/1123589747693060106/image-2.png "User's FORM")


> **Step 3:** From the top left side goto `Runtime` and click the `Run all` button and `READ THE WARNING CAREFULLY` then press the `Run anyway` button and wait for the program to start for the first time. (shown in the image below)

![image-3.png](https://cdn.discordapp.com/attachments/806892281360809985/1123589747961503844/image.png "Running the progeram")
![image-3.png](https://cdn.discordapp.com/attachments/806892281360809985/1123591034488750090/image-2_1.png "Running the progeram")

- *Note 1: Have faith! No information is taken from the user and no data is stored anywhere.*
- *Note 2: It will take some time to load the program first time.*
- *Note 3: It will take some additional time search for the music metadata on spotify and google if `ADD_MUSIC_METADATA` is selected.*
- *Note 4: There should be no errors and the download should automatically start after successful process.*
- *Note 5: If any error occurs, then send me a screenshot of the error with the link to the video.*


> **Step 4:** If you want to download more than one audio files, then, after the first download, update the link to the 2nd video in the `User's FORM` and then from the `Runtime` click the `Run after` button. Continue doing the same for any other videos. (shown in the image in **Step 3**)

# Pre-requisite

In [ ]:
# @title Modules { display-mode: "form" }
from IPython.core.getipython import get_ipython

if "google.colab" in str(get_ipython()):
    %pip install --upgrade "ipywidgets<8.0"  &> /dev/null
    # %pip install --upgrade pytube  &> /dev/null
    %pip install --upgrade mutagen  &> /dev/null
    %pip install --upgrade "requests==2.27.1"  &> /dev/null
    %pip install --upgrade ffmpeg-python  &> /dev/null
    # temporary bugfix for pytube
    %pip install --upgrade "git+https://github.com/alimussifar/pytube"  &> /dev/null

In [ ]:
# @title Custom errors { display-mode: "form" }
class MissingValueError(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)


class InvalidLinkError(Exception):
    def __init__(self, *args: object) -> None:
        super().__init__(*args)

In [ ]:
# @title Helping functions { display-mode: "form" }
import re


def is_valid_link(video_url: str) -> bool:
    # youtube url regex
    pattern = (
        r"^(https?://)?(www\.|m\.)?(youtube|youtu|youtube-nocookie)\.(com|be)/"
        r"(watch\?v=|embed/|v/|.+\?v=)?([^&=%\?]{11})"
    )
    # pattern = (r"(?:v=|\/)([0-9A-Za-z_-]{11}).*")
    regex: re.Pattern[str] = re.compile(pattern)
    return regex.search(video_url) is not None

### FORM: YouTube video link

In [ ]:
# # @title FORM: YouTube Video Link { display-mode: "form" }
# from IPython.core.getipython import get_ipython

# if not "google.colab" in str(get_ipython()):
#     from ipywidgets import widgets

#     video_link_label = widgets.Label("YouTube video link (required): ")
#     video_link = widgets.Text(
#         value="https://www.youtube.com/watch?v=vxgO1fQZhHM",
#         placeholder="e.g.: https://youtube.com/watch?v=h5Vb4uM5Hl0",
#         # description="",
#         disabled=False,
#     )
#     widgets.HBox((video_link_label, video_link))

##### Form validation

In [ ]:
# # @title { display-mode: "form" }
# if not bool(video_link.value):
#     raise MissingValueError("YouTube video link is missing in the form.")

In [ ]:
# # @title { display-mode: "form" }
# if not is_valid_link(video_link.value):
#     raise InvalidLinkError(f"Link '{video_link.value}' is invalid in the form.")

## Get YouTube video information and audio

In [ ]:
# @title YouTube video information and audio class { display-mode: "form" }
import os
from dataclasses import dataclass
from typing import Any, Optional

import ffmpeg
from pytube import YouTube


@dataclass
class YouTubeAudioDownloader:
    """Class for downloading audio from YouTube videos."""

    def _on_progress(self, *args: Any, **kwargs: Any) -> None:
        """Callback function to handle download progress updates."""
        return

    def _on_complete(self, *args: Any, **kwargs: Any) -> None:
        """Callback function to handle download completion."""
        return

    def get_audio(
        self, video_url: str, output_file: Optional[str] = None
    ) -> Optional[str]:
        """Download audio from a YouTube video.

        Parameters
        ----------
        video_url : str
            URL of the YouTube video.
        output_file : str, optional
            Output file path for the downloaded audio, by default None.

        Returns
        -------
        str or None
            Path to the downloaded audio file in M4A format, or None if download fails.
        """

        def convert_to_m4a(filename: str) -> str:
            """Convert a video file to M4A format.

            Parameters
            ----------
            filename : str
                Path to the input video file.

            Returns
            -------
            str
                Path to the output M4A audio file.
            """
            file_name, ext = os.path.splitext(filename)
            output_file = file_name + ".m4a"

            if os.path.exists(output_file):
                os.remove(output_file)

            ffmpeg.input(file_name + ext).output(output_file).run()
            os.remove(filename)

            return output_file

        self.video = YouTube(video_url, self._on_progress, self._on_complete)
        print(self.video.author, self.video.title)

        audio_streams = self.video.streams.filter(
            type="audio", mime_type="audio/mp4"
        ).order_by("abr")
        stream = audio_streams.last()

        # FEATURE: users can add custom download path/filename

        if stream:
            filename = os.path.basename(stream.download())

            return convert_to_m4a(filename)

        return

## Audio metadata class

In [ ]:
# @title # Audio metadata class { display-mode: "form", run: "auto" }
import os
from dataclasses import asdict, dataclass, field
from enum import Enum
from typing import Any, Optional, Union

from mutagen import id3, mp4


class AudioType(Enum):
    MP3 = "mp3"
    MP4 = "mp4"


class Compilation(Enum):
    NO = "0"
    YES = "1"


class MediaType(Enum):
    MOVIE_OLD = "0"
    MUSIC = "1"
    AUDIOBOOK = "2"
    WHACKED_BOOKMARK = "5"
    MUSIC_VIDEO = "6"
    MOVIE = "9"
    TV_SHOW = "10"
    BOOKLET = "11"
    RINGTONE = "14"
    PODCAST = "21"
    ITUNES_U = "23"


class Rating(Enum):
    NONE = "0"
    EXPLICIT = "1"
    CLEAN = "2"
    EXPLICIT_OLD = "4"


@dataclass()
class Metadata:
    """Metadata class for audio files.

    Attributes
    ----------
    audio : Optional[Union[mp4.MP4, id3.ID3]]
        The audio file's metadata object (MP4 or ID3).
    title : Optional[str]
        The title of the audio.
    album : Optional[str]
        The album of the audio.
    album_artist : Optional[str]
        The album artist of the audio.
    artist : Optional[str]
        The artist of the audio.
    track : Optional[tuple[int, int]]
        The track number and total tracks of the audio.
    date : Optional[str]
        The date of the audio.
    genre : Optional[str]
        The genre of the audio.
    composer : Optional[str]
        The composer of the audio.
    disk : tuple[int, int]
        The disk number and total disks of the audio.
    compilation : Optional[int]
        The compilation flag of the audio.
    gapless_playback : Optional[int]
        The gapless playback flag of the audio.
    rating : int
        The rating of the audio.
    media_type : int
        The media type of the audio.
    copyright : Optional[str]
        The copyright information of the audio.
    account_id : Optional[str]
        The account ID of the audio.
    purchase_date : Optional[str]
        The purchase date of the audio.
    sort_name : Optional[str]
        The sorted name of the audio.
    sort_album : Optional[str]
        The sorted album name of the audio.
    sort_album_artist : Optional[str]
        The sorted album artist name of the audio.
    sort_artist : Optional[str]
        The sorted artist name of the audio.
    sort_composer : Optional[str]
        The sorted composer name of the audio.
    lyrics : Optional[str]
        The lyrics of the audio.
    album_art : Optional[Union[str, id3.APIC, mp4.MP4Cover]]
        The album art of the audio.
    comment : Optional[str]
        The comment of the audio.

    Methods
    -------
    add_to_m4a(audio: str, autosave: bool = False) -> None
        Add the metadata to an M4A audio file.
    add_to_mp3(audio: str, autosave: bool = False) -> None
        Add the metadata to an MP3 audio file.
    """

    audio: Optional[Union[mp4.MP4, id3.ID3]] = field(default=None)
    title: Optional[str] = field(default=None)
    album: Optional[str] = field(default=None)
    album_artist: Optional[str] = field(default=None)
    artist: Optional[str] = field(default=None)
    track: Optional[tuple[int, int]] = field(default=None)
    date: Optional[str] = field(default=None)
    genre: Optional[str] = field(default=None)
    composer: Optional[str] = field(default=None)
    disk: tuple[int, int] = field(default_factory=lambda: (1, 1))
    compilation: Optional[int] = field(default=None)
    gapless_playback: Optional[int] = field(default=None)
    rating: int = field(default=int(Rating.NONE.value))
    media_type: int = field(default=int(MediaType.MUSIC.value))
    copyright: Optional[str] = field(default=None)
    account_id: Optional[str] = field(default=None)
    purchase_date: Optional[str] = field(default=None)
    sort_name: Optional[str] = field(default=None)
    sort_album: Optional[str] = field(default=None)
    sort_album_artist: Optional[str] = field(default=None)
    sort_artist: Optional[str] = field(default=None)
    sort_composer: Optional[str] = field(default=None)
    lyrics: Optional[str] = field(default=None)
    album_art: Optional[Union[str, id3.APIC, mp4.MP4Cover]] = field(default=None)
    comment: Optional[str] = field(default=None)

    @staticmethod
    def _map_metadata(key: str, type: AudioType) -> Any:
        """Map the metadata key to the corresponding tag and metadata object.

        Parameters
        ----------
        key : str
            The metadata key.
        type : AudioType
            The audio type (MP4 or MP3).

        Returns
        -------
        Any
            The corresponding tag and metadata object.
        """
        if key not in (
            "title",
            "album",
            "album_artist",
            "artist",
            "track",
            "date",
            "genre",
            "composer",
            "disk",
            "compilation",
            "gapless_playback",
            "rating",
            "media_type",
            "copyright",
            "account_id",
            "purchase_date",
            "sort_name",
            "sort_album",
            "sort_album_artist",
            "sort_artist",
            "sort_composer",
            "lyrics",
            "album_art",
            "comment",
        ):
            return

        return {
            "title": {
                AudioType.MP4: "\xa9nam",
                AudioType.MP3: ("TIT2", id3.TIT2),
            },
            "album": {
                AudioType.MP4: "\xa9alb",
                AudioType.MP3: ("TALB", id3.TALB),
            },
            "album_artist": {
                AudioType.MP4: "aART",
                AudioType.MP3: ("TPE2", id3.TPE2),
            },
            "artist": {
                AudioType.MP4: "\xa9ART",
                AudioType.MP3: ("TPE1", id3.TPE1),
            },
            "track": {
                AudioType.MP4: "trkn",  # (track_number, total_tracks)
                AudioType.MP3: ("TRCK", id3.TRCK),
            },
            "date": {
                AudioType.MP4: "\xa9day",
                AudioType.MP3: ("TDRC", id3.TDRC),
            },
            "genre": {
                AudioType.MP4: "\xa9gen",
                AudioType.MP3: ("TCON", id3.TCON),
            },
            "composer": {
                AudioType.MP4: "\xa9wrt",
                AudioType.MP3: ("TCOM", id3.TCOM),
            },
            "disk": {
                AudioType.MP4: "disk",  # (track_number, total_tracks)
                AudioType.MP3: ("TPOS", id3.TPOS),
            },
            "compilation": {
                AudioType.MP4: "cpil",
                AudioType.MP3: ("TCMP", id3.TCMP),
            },
            "gapless_playback": {
                AudioType.MP4: "pgap",
                AudioType.MP3: None,
            },
            "rating": {
                AudioType.MP4: "rtng",
                AudioType.MP3: None,
            },
            "media_type": {
                AudioType.MP4: "stik",
                AudioType.MP3: None,
            },
            "copyright": {
                AudioType.MP4: "cprt",
                AudioType.MP3: ("TCOP", id3.TCOP),
            },
            "account_id": {
                AudioType.MP4: "apID",
                AudioType.MP3: None,
            },
            "purchase_date": {
                AudioType.MP4: "purd",
                AudioType.MP3: None,
            },
            "sort_name": {
                AudioType.MP4: "sonm",
                AudioType.MP3: ("TSOT", id3.TSOT),
            },
            "sort_album": {
                AudioType.MP4: "soal",
                AudioType.MP3: ("TSOA", id3.TSOA),
            },
            "sort_album_artist": {
                AudioType.MP4: "soaa",
                AudioType.MP3: ("TSO2", id3.TSO2),
            },
            "sort_artist": {
                AudioType.MP4: "soar",
                AudioType.MP3: ("TSOP", id3.TSOP),
            },
            "sort_composer": {
                AudioType.MP4: "soco",
                AudioType.MP3: ("TSOC", id3.TSOC),
            },
            "lyrics": {
                AudioType.MP4: "\xa9lyr",
                AudioType.MP3: ("USLT", id3.USLT),
            },
            "album_art": {
                AudioType.MP4: "covr",
                AudioType.MP3: ("APIC", id3.APIC),
            },
            "comment": {
                AudioType.MP4: "\xa9cmt",
                AudioType.MP3: ("COMM", id3.COMM),
            },
        }[key][type]

    def add_to_m4a(self, audio: str, autosave: bool = False) -> None:
        """Add the metadata to an M4A audio file.

        Parameters
        ----------
        audio : str
            The path to the M4A audio file.
        autosave : bool, optional
            Whether to save the changes to the audio file automatically, by default False.
        """
        if self.album_art and isinstance(self.album_art, str):
            # IMPROVEMENT: instead of loading local image, get the bytes from the url
            # then use the bytes directly to set the album_art

            # open image as bytes
            album_art = self.album_art
            with open(self.album_art, "rb") as image:
                self.album_art = mp4.MP4Cover(data=image.read())
            os.remove(album_art)

        audio_ = mp4.MP4(audio)

        for key, value in asdict(self).items():
            if value is not None and self._map_metadata(key, AudioType.MP4) is not None:
                audio_[self._map_metadata(key, AudioType.MP4)] = [value]

        if autosave:
            audio_.save()
        return

    def add_to_mp3(self, audio: str, autosave: bool = False) -> None:
        """
        Add the metadata to an MP3 audio file.

        Parameters
        ----------
        audio : str
            The path to the MP3 audio file.
        autosave : bool, optional
            Whether to save the changes to the audio file automatically, by default False.
        """
        if self.album_art and isinstance(self.album_art, str):
            # IMPROVEMENT: instead of loading local image, get the bytes from the url
            # then use the bytes directly to set the album_art

            # open image as bytes
            album_art = self.album_art
            with open(self.album_art, "rb") as image:
                self.album_art = id3.APIC(
                    encoding=3, mime="image/jpeg", type=0, data=image.read()
                )
            os.remove(album_art)

        audio_ = id3.ID3(audio)

        for key, value in asdict(self).items():
            if value is None or self._map_metadata(key, AudioType.MP3) is None:
                continue

            tag, metadata = self._map_metadata(key, AudioType.MP3)

            if key in ("track", "disk"):
                # track_disk_number : str/str
                audio_[tag] = metadata(encoding=3, text="/".join(map(str, value)))
            elif key in ("album_art",):
                # album_art : encoding=3, mime='image/jpeg', type=0, desc='Cover',
                # data=open('path/to/album_art.jpg', 'rb').read()
                # NOTE: album cover is added successfully, windows cannot display that.
                audio_[tag] = value
            elif key in ("comment",):
                # comment : encoding=3, desc='sort_name', text='Sort Name'
                # NOTE: comment is added successfully, iTunes cannot display that.
                audio_[tag] = metadata(encoding=3, desc=value, text=value)
            elif isinstance(
                metadata(), (id3.TXXX, id3.TSOT, id3.TSOA, id3.TSO2, id3.TSOP, id3.TSOC)
            ):
                # encoding=3, desc='sort_name', text='Sort Name'
                audio_[tag] = metadata(encoding=3, desc=key, text=value)
            elif key in ("lyrics",):
                # lyrics : encoding=3, text='Sort Name'
                audio_[tag] = metadata(encoding=3, text=value)
            else:
                audio_[tag] = metadata(encoding=3, text=str(value))

        if autosave:
            audio_.save()
        return

## Get audio metadata from Spotify

In [ ]:
# @title Spotify authentication { display-mode: "form" }
import base64
import json
import os
import re
from dataclasses import dataclass, field
from datetime import datetime
from io import BytesIO
from typing import Any, Optional

import requests
from bs4 import BeautifulSoup
from PIL import Image
from requests import Response

# from .metadata import Metadata


@dataclass
class SpotifyAPI:
    """Class representing the Spotify API."""

    auth: dict[str, Any] = field(default_factory=lambda: {})

    def is_authenticated(self, filename: str = "auth.json") -> bool:
        """Check if the user is authenticated.

        Parameters
        ----------
        filename : str, optional
            The name of the authentication file, by default "auth.json"

        Returns
        -------
        bool
            True if authenticated, False otherwise.
        """
        if not os.path.exists(filename):
            # create a auth.json file and return false
            with open(filename, "w") as jsonfile:
                json.dump(self.auth, jsonfile)
            return False

        with open(filename, "rb") as jsonfile:
            # check if the auth has expired
            self.auth = json.load(jsonfile)
            if (
                not self.auth
                or datetime.now().timestamp() > self.auth["authorize_after"]
            ):
                return False

        return True

    def authenticate(self, filename: str = "auth.json") -> None:
        """Authenticate the user.

        Parameters
        ----------
        filename : str, optional
            The name of the authentication file, by default "auth.json"
        """
        now = datetime.now().timestamp()

        # Spotify - App Tester - App Token
        B64ClientID = base64.standard_b64encode(b"91a704941f39447980874befd4221bb6:")
        B64AuthToken = (
            f"{B64ClientID.decode()}MmEwYjBjMDY3MTZkNDg0MWIxNTc2NmI3YWI4MzE5Njk="
        )

        ENDPOINT = "https://accounts.spotify.com/api/token"
        form = {"grant_type": "client_credentials"}
        headers = {
            "Authorization": f"Basic {B64AuthToken}",
            "Content-Type": "application/x-www-form-urlencoded",
        }

        self.auth = requests.post(ENDPOINT, params=form, headers=headers).json()
        self.auth["authorize_after"] = now + self.auth["expires_in"]

        with open(filename, "w") as jsonfile:
            json.dump(self.auth, jsonfile, indent=2)
        return

    def search(
        self,
        *query: str,
        type_: str = "track",  # REQUIRED: Values: "album", "artist", "playlist", "track", "show", "episode", "audiobook"
        market: Optional[str] = None,
        limit: int = 5,  # Value Range: 0 - 50
        offset: int = 0,  # Value Range: 0 - 1000
        include_external: Optional[str] = None,  # Values: "audio"
    ) -> Any:
        """Search for tracks, albums, artists, playlists, shows, episodes, or audiobooks on Spotify.

        Parameters
        ----------
        query : str
            The search query.
        type_ : str, optional
            The type of item to search for, by default "track".
        market : str, optional
            An ISO 3166-1 alpha-2 country code to limit the search results, by default None.
        limit : int, optional
            The maximum number of items to return, by default 5.
        offset : int, optional
            The index of the first item to return, by default 0.
        include_external : str, optional
            Whether to include external audio content in the search results, by default None.

        Returns
        -------
        Any
            The search results.
        """
        ENDPOINT = "https://api.spotify.com/v1/search"
        params = {
            # "q": f"{video.title}, {video.author}",  # REQUIRED
            "q": ", ".join(map(str, query)),  # REQUIRED
            "type": type_,  # REQUIRED: Values: "album", "artist", "playlist", "track", "show", "episode", "audiobook"
            "market": market,
            "limit": limit,  # Value Range: 0 - 50
            "offset": offset,  # Value Range: 0 - 1000
            "include_external": include_external,  # Values: "audio"
        }
        headers = {
            "Authorization": f"{self.auth.get('token_type')} {self.auth.get('access_token')}",
            "Content-Type": "application/json",
        }

        self.res = requests.get(ENDPOINT, params=params, headers=headers).json()
        return self.res

    def to_metadata(self) -> Optional[Metadata]:
        """Convert the Spotify API response to metadata.

        Returns
        -------
        Metadata, optional
            The converted metadata or None if no tracks found.
        """

        def request(url: str, params=None) -> Response:
            """Send a GET request to the specified URL.

            Parameters
            ----------
            url : str
                The URL to send the request to.
            params : dict, optional
                The query parameters for the request, by default None.

            Returns
            -------
            Response
                The response object.
            """
            headers = {
                "User-Agent": (
                    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                    "(KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
                )
                # "User-Agent": "Mozilla/5.0",
            }

            return requests.get(url, params, headers=headers)

        def music_search(*args: str) -> Optional[str]:
            """Perform a music search using Google search.

            Parameters
            ----------
            args : str
                The search query.

            Returns
            -------
            str, optional
                The URL of the music search result or None if not found.
            """
            # make a request to website
            query = ", ".join(map(str, args))
            params = {"q": f"site:music.apple.com {query} "}
            res = request("https://google.com/search", params)
            soup = BeautifulSoup(res.content, "html.parser")

            # Find the div with id 'rso' within the specified hierarchy
            div_rso = soup.select_one(
                "body > div#main > div#cnt > div#rcnt > div#center_col > div#res > div#search > div > div#rso"
            )

            # Find all anchor tags (a) within the 'div_rso' that match the regex pattern
            if div_rso:
                # Create a regular expression pattern
                pattern = re.compile(r"^https://music.apple.com/[a-z]{2}/album/\S+/\d+")
                return sorted([a["href"] for a in div_rso.find_all("a", href=pattern)])[
                    -1
                ]
            return

        def scrape_data(url: str) -> dict[str, Any]:
            """Scrape data from a specified URL.

            Parameters
            ----------
            url : str
                The URL to scrape data from.

            Returns
            -------
            dict[str, Any]
                The scraped data.
            """
            data = {}
            # make a request to website
            res = request(url)
            soup = BeautifulSoup(res.content, "html.parser")
            div_heading = soup.select_one(
                (
                    "div#scrollable-page > main > div.content-container > div.section > div.section-content > "
                    "div.container-detail-header > div.headings > div.headings__metadata-bottom"
                )
            )
            div_footer = soup.select_one(
                (
                    "div#scrollable-page > main > div.content-container > div.section > div.section-content > "
                    "div.tracklist-footer > div.footer-body > p.description"
                )
            )

            if div_heading:
                data["genre"] = div_heading.text.split(" · ")[0].title()
            if div_footer:
                data["copyright"] = div_footer.text.split("\n")[-1].strip()

            return data

        def get_artists(artists: Any) -> str:
            """Get the names of the artists.

            Parameters
            ----------
            artists : Any
                The artist data.

            Returns
            -------
            str
                The names of the artists.
            """
            artists = [artist.get("name") for artist in artists]
            if len(artists) > 1:
                artists_ = ", ".join(artists[:-1])
                artists_ = f"{artists_} & {artists[-1]}"
                return artists_
            return artists.pop()

        def get_album_art(album_arts: list[dict[str, Any]], filename: str) -> str:
            """Get the album art.

            Parameters
            ----------
            album_arts : list[dict[str, Any]]
                The album art data.
            filename : str
                The filename to save the album art.

            Returns
            -------
            str
                The filename of the saved album art.
            """
            # IMPROVEMENT: instead of saving the image, store it as bytes and return
            # then use the bytes directly to set the album_art
            res = requests.get(str(album_arts[0].get("url")))
            Image.open(BytesIO(res.content)).save(f"{filename}.jpg")
            return f"{filename}.jpg"

        if not self.res.get("tracks"):
            return

        items = self.res.get("tracks").get("items")
        item_1 = items[0]

        filename: str = item_1.get("album").get("id")

        title: str = item_1.get("name")
        album_name: str = item_1.get("album").get("name")
        album_type: str = item_1.get("album").get("album_type")
        album_artist: str = get_artists(item_1.get("album").get("artists"))

        if album_type == "single":
            album: str = f"{album_name} - {album_type.title()}"
        elif album_type == "ep":
            album: str = f"{album_name} - {album_type.upper()}"
        else:
            album: str = album_name

        data: Optional[Any] = music_search(album_artist, album_name)
        if data:
            data = scrape_data(data)

        artist: str = get_artists(item_1.get("artists"))
        track: int = item_1.get("track_number")
        total_tracks: int = item_1.get("album").get("total_tracks")
        date: str = item_1.get("album").get("release_date")
        genre: Optional[str] = data.get("genre") if data else None
        composer: Optional[str] = None  # TODO: NOT IMPLEMENTED
        disk: int = item_1.get("disc_number")
        compilation: Optional[int] = 1 if album_type == "compilation" else None
        gapless_playback = None
        rating = int(item_1.get("explicit"))
        # media_type = ...
        copyright: Optional[str] = data.get("copyright") if data else None
        account_id = "alimussifar@icloud.com"
        purchase_date: Optional[str] = None  # TODO: NOT IMPLEMENTED
        # sort_name = ...
        # sort_album = ...
        # sort_album_artist = ...
        # sort_artist = ...
        # sort_composer = ...
        lyrics: Optional[str] = None  # TODO: NOT IMPLEMENTED
        album_art: str = get_album_art(item_1.get("album").get("images"), filename)
        comment = (
            "Metadata collected from SpotifyAPI and added via the "
            "python package named 'Mutagen'"
        )

        return Metadata(
            title=title,
            album=album,
            album_artist=album_artist,
            artist=artist,
            track=(track, total_tracks),
            date=date,
            genre=genre,
            composer=composer,
            disk=(disk, disk),
            compilation=compilation,
            gapless_playback=gapless_playback,
            rating=rating,
            media_type=1,
            copyright=copyright,
            account_id=account_id,
            purchase_date=purchase_date,
            sort_name=title,
            sort_album=album,
            sort_album_artist=album_artist,
            sort_artist=artist,
            sort_composer=composer,
            lyrics=lyrics,
            album_art=album_art,
            comment=comment,
        )

# Main Section

In [ ]:
# @title ## User's FORM { display-mode: "form", run: "auto" }

# @markdown ### YouTube video link (*required)
# @markdown > Example: `https://www.youtube.com/watch?v=cSLAO7zxS2M` or any valid youtube audio/video links
VIDEO_URL = ""  # @param { type: "string" }

# @markdown ### Search and add metadata (e.g. song name, album, artist, etc.) to the audio file?
ADD_MUSIC_METADATA = False  # @param { type: "boolean" }

### Form validation

In [ ]:
# @title { display-mode: "form" }
if not bool(VIDEO_URL):
    raise MissingValueError("YouTube video link is missing in the form.")

In [ ]:
# @title { display-mode: "form" }
if not is_valid_link(str(VIDEO_URL)):
    raise InvalidLinkError(f"Link '{VIDEO_URL}' is invalid in the form.")

# Start process

In [ ]:
# @title Process user information { display-mode: "form", run: "auto" }
from IPython.core.getipython import get_ipython

print("[INFO] Getting YouTube data to download.")
ytad = YouTubeAudioDownloader()
audio_path = ytad.get_audio(VIDEO_URL)
print(f"[INFO] Found audio: `{audio_path}`")

if not ADD_MUSIC_METADATA:
    print("[INFO] Metadata skipped.")
else:
    print("[INFO] Collecting metadata, please wait.")

    spotify = SpotifyAPI()
    if not spotify.is_authenticated():
        spotify.authenticate()

    print("[INFO] Found audio metadata from SpotifyAPI.")
    y = spotify.search(
        ytad.video.author,
        ytad.video.title,
        # market="US",
        limit=1,
    )
    # print(y)
    print("[INFO] Adding metadata to audio.")
    metadata = spotify.to_metadata()
    if metadata and audio_path:
        print(metadata)
        metadata.add_to_m4a(audio_path, autosave=True)

if "google.colab" in str(get_ipython()):
    from google.colab import files

    print("[INFO] Downloading audio.")
    files.download(audio_path)

print("[INFO] Program ended.")